# Inerpretation-Net Training

# Experiment 1: I-Net Performance for Different Algebras and Complexities
# Experiment 2: I-Net Performance Comparison for λ-Nets with Different Training Levels
# Experiment 3: I-Net Performance Comparison Different Training Data Sizes

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'data': {
        'd': 5, #degree
        'n': 1, #number of variables
        'sparsity': None, #calculated automatically later
        'sample_sparsity': None,
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform', 'beta', 'Gamma', 'laplace'
        'a_max': 1,
        'a_min': -1,
        'lambda_nets_total': 10000,
        'noise': 0,
        'noise_distrib': 'normal', #'normal', 'uniform', 'beta', 'Gamma', 'laplace'
        
        'same_training_all_lambda_nets': False,

        'fixed_seed_lambda_training': True,
        'fixed_initialization_lambda_training': False,
        'number_different_lambda_trainings': 1,
    },
    'lambda_net': {
        'epochs_lambda': 200,
        'batch_lambda': 64,
        'dropout': 0,
        'lambda_network_layers': [2*'sparsity'],
        'optimizer_lambda': 'adam',
        'number_of_lambda_weights': None,
        'lambda_dataset_size': 1000,
    },
    'i_net': {
        'optimizer': 'custom',#adam
        'dropout': 0.25,
        'epochs': 100,
        'early_stopping': True,
        'batch_size': 512,
        'dense_layers': [512, 1024],
        'convolution_layers': None,
        'lstm_layers': None,
        'interpretation_dataset_size': 10000,
        
        'interpretation_net_output_monomials': None, #(None, int)
        'interpretation_net_output_shape': None, #calculated automatically later
        
        'evaluate_with_real_function': False,
        'consider_labels_training': False,
        
        'seed_in_inet_training': False,
              
        'data_reshape_version': None, #default to 2 options:(None, 0,1 2)
        'nas': False,
        'nas_type': None, #options:(None, 'CNN', 'LSTM', 'CNN-LSTM', 'CNN-LSTM-parallel')      
        'nas_trials': 0,
    },
    'evaluation': {   
        'inet_holdout_seed_evaluation': False,
        
        #set if multi_epoch_analysis should be performed
        'multi_epoch_analysis': True,
        'each_epochs_save_lambda': 20,
        'epoch_start': 0, #use to skip first epochs in multi_epoch_analysis
        
        #set if samples analysis should be performed
        'samples_list': None,#[100, 500, 750, 1000, 2500, 5000, 7500, 10000, 15000, 20000, 25000, 28125] 
       
        'random_evaluation_dataset_size': 500,
    },
    'computation':{
        'n_jobs': 11,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}

## Imports

In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [3]:
#######################################################################################################################################
##################################################### IMPORT LIBRARIES ################################################################
#######################################################################################################################################
from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *


from itertools import product       
from tqdm import tqdm_notebook as tqdm
import pickle
import numpy as np
import pandas as pd
import scipy as sp
import timeit
import psutil

from functools import reduce
from more_itertools import random_product 
from sklearn.preprocessing import Normalizer


import os
import shutil

import logging

from prettytable import PrettyTable
import colored
import math

import time
from datetime import datetime
from collections.abc import Iterable


from joblib import Parallel, delayed

from scipy.integrate import quad

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_absolute_error, r2_score
from similaritymeasures import frechet_dist, area_between_two_curves, dtw
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.utils import plot_model
from IPython.display import Image

import keras.backend as K
from livelossplot import PlotLossesKerasTF
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from matplotlib import pyplot as plt
import seaborn as sns


import tensorflow as tf
import random 


import warnings


from IPython.display import display, Math, Latex



In [4]:
#######################################################################################################################################
################################################### VARIABLE ADJUSTMENTS ##############################################################
#######################################################################################################################################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n] 


n_jobs = min((epochs_lambda//each_epochs_save_lambda+1, n_jobs)) if multi_epoch_analysis else min(len(samples_list), n_jobs) if samples_list!=None else 1

each_epochs_save_lambda = each_epochs_save_lambda if multi_epoch_analysis else epochs_lambda
epochs_save_range_lambda = range(epoch_start//each_epochs_save_lambda, epochs_lambda//each_epochs_save_lambda) if each_epochs_save_lambda == 1 else range(epoch_start//each_epochs_save_lambda, epochs_lambda//each_epochs_save_lambda+1) if multi_epoch_analysis else range(1,2)

#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''

logging.getLogger('tensorflow').disabled = True

sns.set_style("darkgrid")
#np.set_printoptions(suppress=True)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if int(tf.__version__[0]) >= 2:
    tf.random.set_seed(RANDOM_SEED)
else:
    tf.set_random_seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [5]:
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################
config['evaluation']['each_epochs_save_lambda'] = each_epochs_save_lambda

config['data']['sparsity'] = nCr(config['data']['n']+config['data']['d'], config['data']['d'])
config['data']['sample_sparsity'] = config['data']['sparsity'] if config['data']['sample_sparsity'] == None else config['data']['sample_sparsity']

config['i_net']['interpretation_net_output_shape'] = config['data']['sparsity'] if config['i_net']['interpretation_net_output_monomials'] is None else 2*config['i_net']['interpretation_net_output_monomials']

transformed_layers = []
for layer in config['lambda_net']['lambda_network_layers']:
    if type(layer) == str:
        transformed_layers.append(layer.count('sparsity')*config['data']['sparsity'])
    else:
        transformed_layers.append(layer)
config['lambda_net']['lambda_network_layers'] = transformed_layers

layers_with_input_output = list(flatten([[config['data']['n']], config['lambda_net']['lambda_network_layers'], [1]]))
number_of_lambda_weights = 0
for i in range(len(layers_with_input_output)-1):
    number_of_lambda_weights += (layers_with_input_output[i]+1)*layers_with_input_output[i+1]  
config['lambda_net']['number_of_lambda_weights'] = number_of_lambda_weights
    
#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])



initialize_LambdaNet_config_from_curent_notebook(config)
initialize_metrics_config_from_curent_notebook(config)
initialize_utility_functions_config_from_curent_notebook(config)
initialize_InterpretationNet_config_from_curent_notebook(config)


#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(path_type='interpretation_net'))
create_folders_inet()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [6]:
print(path_identifier_interpretation_net_data)

print(path_identifier_lambda_net_data)


inet_dense[512, 1024]convNonelstmNonedrop0.25e100b512_customlambd_nets_10000_12-200e64b_adam_train_1000_variables_1_degree_5_sparsity_6_amin_-1_amax_1_xmin_0_xmax_1_xdistrib_uniform_noise_normal_0_differentX_1-FixedSeed_NoFixedInitialization_42
lambd_nets_10000_12-200e64b_adam_train_1000_variables_1_degree_5_sparsity_6_amin_-1_amax_1_xmin_0_xmax_1_xdistrib_uniform_noise_normal_0_differentX_1-FixedSeed_NoFixedInitialization_42


In [7]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


## Utility functions

### Generate List of Monomial Identifiers

In [8]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)


layers_with_input_output = list(flatten([[n], lambda_network_layers, [1]]))
number_of_lambda_weights = 0
for i in range(len(layers_with_input_output)-1):
    number_of_lambda_weights += (layers_with_input_output[i]+1)*layers_with_input_output[i+1]

  0%|          | 0/6 [00:00<?, ?it/s]

List length: 6
Number of monomials in a polynomial with 1 variables and degree 5: 6
Sparsity: 6
['0', '1', '2', '3', '4', '5']


  0%|          | 0/6 [00:00<?, ?it/s]

List length: 6
Number of monomials in a polynomial with 1 variables and degree 5: 6
Sparsity: 6
['0', '1', '2', '3', '4', '5']


## Load Data and Generate Datasets

In [9]:
def load_lambda_nets(index):
    
    if psutil.virtual_memory().percent > 80:
        raise SystemExit("Out of RAM!")
    
    path = './data/weights/' + 'weights_' + path_identifier_lambda_net_data + '/weights_' + path_identifier_lambda_net_data + '_epoch_' + str(index).zfill(3) + '.txt'

    
    weight_data = pd.read_csv(path, sep=",", header=None)
    weight_data = weight_data.sort_values(by=0).sample(frac=1, random_state=RANDOM_SEED)
    weight_data = weight_data.sort_values(by=0).sample(n=interpretation_dataset_size, random_state=RANDOM_SEED)

    lambda_nets = []
    for _, row in weight_data.iterrows():
        lambda_net = LambdaNet(row.values)
        lambda_nets.append(lambda_net)
        
    lambda_net_dataset = LambdaNetDataset(lambda_nets)
        
    return lambda_net_dataset
    

In [10]:
#LOAD DATA

parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
lambda_net_dataset_list = parallel(delayed(load_lambda_nets)((i+1)*each_epochs_save_lambda if each_epochs_save_lambda==1 else i*each_epochs_save_lambda if i > 1 else each_epochs_save_lambda if i==1 else 1) for i in epochs_save_range_lambda)  
del parallel

lambda_net_dataset = lambda_net_dataset_list[-1]


[Parallel(n_jobs=11)]: Using backend MultiprocessingBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   2 out of  11 | elapsed:  2.5min remaining: 11.2min
[Parallel(n_jobs=11)]: Done   6 out of  11 | elapsed:  2.6min remaining:  2.1min
[Parallel(n_jobs=11)]: Done  11 out of  11 | elapsed:  2.7min finished


## Data Inspection

In [11]:
lambda_net_dataset.as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36
3745,1373158606,0.124,0.523,0.732,0.206,0.632,-0.375,0.073,1.101,-0.828,-0.029,4.862,-3.439,0.025,-0.284,-0.622,1.387,-1.380,0.505,-0.014,-0.350,0.198,0.126,0.730,0.629,0.166,-0.459,0.103,1.405,0.745,-0.136,0.000,0.000,-0.198,-0.126,-0.003,0.033,-0.166,0.000,-0.103,-0.775,-0.229,0.000,-0.392,-0.554,-0.313,-0.353,0.663,0.714,-0.067,-0.630,-0.403,0.871,0.403,-0.544,0.057
1710,1373158606,0.363,0.360,0.338,0.885,0.646,-0.916,0.291,1.236,-3.146,6.949,-4.156,0.495,0.025,-0.284,-0.621,1.386,-1.379,0.504,-0.014,-0.350,0.202,0.128,0.677,0.577,0.169,-0.459,0.104,1.257,0.640,-0.136,0.000,0.000,-0.202,-0.128,0.133,0.133,-0.169,0.000,-0.105,-0.582,-0.427,0.000,-0.392,-0.554,-0.314,-0.353,0.617,0.663,-0.069,-0.630,-0.404,0.720,0.438,-0.544,0.133
5836,1373158606,-0.562,-0.713,0.126,0.343,0.139,0.313,-0.576,-0.587,0.843,-4.816,8.913,-4.213,-0.030,-0.551,0.255,-0.570,0.567,-0.208,-0.014,-0.350,0.351,0.181,0.821,0.739,0.273,-0.459,0.128,0.348,0.546,-0.136,0.000,0.000,0.184,0.188,-0.418,-0.549,0.192,0.000,0.189,0.216,0.183,0.000,-0.392,-0.554,-0.471,-0.514,1.070,1.363,-0.245,-0.630,-0.577,-0.066,-0.237,-0.544,-0.179
3837,1373158606,-0.446,-0.557,-0.240,0.527,0.427,0.325,-0.455,-0.593,1.329,-6.954,12.283,-5.709,-0.029,-0.547,0.249,-0.556,0.553,-0.202,-0.014,-0.350,0.356,0.187,0.910,0.826,0.285,-0.459,0.134,0.385,0.552,-0.136,0.000,0.000,0.152,0.155,-0.472,-0.618,0.153,0.000,0.157,0.142,0.148,0.000,-0.392,-0.554,-0.451,-0.489,1.264,1.564,-0.220,-0.630,-0.550,-0.033,-0.220,-0.544,-0.153
6455,1373158606,-0.689,0.097,0.949,0.262,-0.102,0.655,-0.706,0.347,0.537,-1.573,5.012,-2.555,-0.021,-0.426,0.122,-0.274,0.274,-0.101,-0.014,-0.350,-0.015,-0.279,0.944,0.869,0.103,-0.459,-0.318,1.175,0.527,-0.136,0.000,0.000,0.259,0.279,-0.343,-0.659,0.209,0.000,0.317,-0.719,0.201,0.000,-0.392,-0.554,-0.268,-0.564,1.015,1.359,0.015,-0.630,-0.738,0.845,0.148,-0.544,-0.284


In [12]:
lambda_net_dataset.as_pandas().describe()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36
count,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000,10000.000
mean,1373158606.000,0.001,-0.003,0.002,-0.001,-0.003,-0.003,0.004,0.000,-0.070,-0.175,0.703,-0.479,0.004,-0.388,-0.266,0.594,-0.591,0.216,-0.014,-0.350,0.401,0.270,0.507,0.414,0.385,-0.459,0.228,0.712,0.661,-0.136,0.000,0.000,-0.031,-0.055,0.022,-0.017,-0.073,0.000,-0.057,-0.096,-0.029,0.000,-0.392,-0.554,-0.517,-0.608,0.575,0.680,-0.297,-0.630,-0.678,0.102,-0.189,-0.544,0.011
std,0.000,0.572,0.578,0.579,0.577,0.576,0.580,0.525,0.787,2.582,7.808,10.833,4.704,0.026,0.120,0.410,0.915,0.911,0.333,0.000,0.000,0.197,0.208,0.259,0.259,0.210,0.000,0.210,0.351,0.175,0.000,0.000,0.000,0.200,0.220,0.239,0.293,0.235,0.000,0.228,0.417,0.204,0.000,0.000,0.000,0.214,0.293,0.279,0.339,0.315,0.000,0.327,0.462,0.317,0.000,0.204
min,1373158606.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.072,-3.361,-10.936,-45.696,-42.783,-26.015,-0.030,-0.553,-0.629,-0.580,-1.397,-0.211,-0.014,-0.350,-0.363,-0.384,-0.307,-0.366,-0.280,-0.459,-0.435,-0.277,0.270,-0.136,0.000,0.000,-0.556,-0.660,-0.748,-0.839,-0.694,0.000,-0.653,-1.137,-0.753,0.000,-0.392,-0.554,-1.389,-1.760,-0.033,0.170,-1.294,-0.630,-2.149,-1.377,-1.023,-0.544,-0.731
25%,1373158606.000,-0.490,-0.504,-0.505,-0.500,-0.498,-0.505,-0.412,-0.501,-1.367,-2.236,-3.448,-1.458,-0.029,-0.550,-0.621,-0.566,-1.379,-0.206,-0.014,-0.350,0.219,0.127,0.323,0.228,0.211,-0.459,0.104,0.457,0.529,-0.136,0.000,0.000,-0.201,-0.149,-0.063,-0.124,-0.230,0.000,-0.118,-0.459,-0.146,0.000,-0.392,-0.554,-0.608,-0.679,0.399,0.463,-0.397,-0.630,-0.760,-0.121,-0.366,-0.544,-0.164
50%,1373158606.000,0.004,-0.002,0.004,-0.008,-0.006,-0.001,0.002,-0.018,-0.000,0.000,-0.000,-0.000,0.025,-0.293,-0.598,1.334,-1.328,0.486,-0.014,-0.350,0.405,0.252,0.479,0.387,0.364,-0.459,0.188,0.646,0.632,-0.136,0.000,0.000,-0.001,-0.085,0.001,0.001,-0.002,0.000,-0.103,0.016,-0.001,0.000,-0.392,-0.554,-0.477,-0.527,0.538,0.609,-0.255,-0.630,-0.577,0.097,-0.236,-0.544,0.003
75%,1373158606.000,0.501,0.490,0.500,0.502,0.497,0.503,0.432,0.524,0.572,3.243,3.160,1.275,0.025,-0.284,0.253,1.386,0.563,0.504,-0.014,-0.350,0.531,0.406,0.669,0.582,0.521,-0.459,0.365,0.929,0.772,-0.136,0.000,0.000,0.142,0.129,0.207,0.207,0.125,0.000,0.125,0.219,0.131,0.000,-0.392,-0.554,-0.346,-0.354,0.667,0.746,-0.091,-0.630,-0.404,0.282,-0.093,-0.544,0.195
max,1373158606.000,1.000,1.000,1.000,1.000,0.999,0.999,1.045,3.291,17.045,31.388,59.649,18.270,0.026,-0.282,0.259,1.404,0.577,0.511,-0.014,-0.350,1.030,0.925,1.337,1.218,1.063,-0.459,1.054,1.800,1.343,-0.136,0.000,0.000,0.562,0.384,0.428,0.435,0.325,0.000,0.435,0.576,0.338,0.000,-0.392,-0.554,-0.074,-0.230,1.667,1.880,0.992,-0.630,-0.322,1.478,0.876,-0.544,0.451


## Generate Datasets for Interpretation-Net training

In [13]:
#generate train, test and validation data for training

lambda_net_train_dataset_list = []
lambda_net_valid_dataset_list = []
lambda_net_test_dataset_list = []

for lambda_net_dataset in lambda_net_dataset_list:
    
    
    if inet_holdout_seed_evaluation:
        complete_seed_list = list(set(lambda_net_dataset.train_settings_list['seed']))#list(weight_data.iloc[:,1].unique())

        random.seed(RANDOM_SEED)
        test_seeds = random.sample(complete_seed_list, int(len(complete_seed_list)-len(complete_seed_list)/(1/0.9)))
        lambda_net_test_dataset = lambda_net_dataset.get_lambda_nets_by_seed(test_seeds)
        complete_seed_list = list(set(complete_seed_list) - set(test_seeds))#complete_seed_list.remove(test_seeds)
        
        random.seed(RANDOM_SEED)
        valid_seeds = random.sample(complete_seed_list, int(len(complete_seed_list)-len(complete_seed_list)/(1/0.9)))
        lambda_net_valid_dataset = lambda_net_dataset.get_lambda_nets_by_seed(valid_seeds)
        complete_seed_list = list(set(complete_seed_list) - set(valid_seeds))

        train_seeds = complete_seed_list
        lambda_net_train_dataset = lambda_net_dataset.get_lambda_nets_by_seed(train_seeds)       
        
        lambda_net_train_dataset_list.append(lambda_net_train_dataset)
        lambda_net_valid_dataset_list.append(lambda_net_valid_dataset)
        lambda_net_test_dataset_list.append(lambda_net_test_dataset)
        
        del lambda_net_dataset
    else:
        lambda_net_train_with_valid_dataset, lambda_net_test_dataset = split_LambdaNetDataset(lambda_net_dataset, test_split=0.1)
        lambda_net_train_dataset, lambda_net_valid_dataset = split_LambdaNetDataset(lambda_net_train_with_valid_dataset, test_split=0.1)

        lambda_net_train_dataset_list.append(lambda_net_train_dataset)
        lambda_net_valid_dataset_list.append(lambda_net_valid_dataset)
        lambda_net_test_dataset_list.append(lambda_net_test_dataset)
    
        del lambda_net_dataset, lambda_net_train_with_valid_dataset

        
del lambda_net_dataset_list

In [14]:
lambda_net_train_dataset_list[-1].as_pandas().shape

(8100, 56)

In [15]:
lambda_net_valid_dataset_list[-1].as_pandas().shape

(900, 56)

In [16]:
lambda_net_test_dataset_list[-1].as_pandas().shape

(1000, 56)

In [17]:
lambda_net_train_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36
3625,1373158606,-0.903,0.119,0.502,-0.609,-0.264,-0.967,-0.699,-0.624,-0.000,0.000,-0.000,0.000,-0.030,-0.552,0.257,-0.574,0.571,-0.209,-0.014,-0.350,0.445,0.279,0.390,0.300,0.371,-0.459,0.228,0.457,0.635,-0.136,0.000,0.000,0.203,0.206,0.000,0.001,0.216,0.000,0.206,0.257,0.209,0.000,-0.392,-0.554,-0.535,-0.551,0.371,0.426,-0.299,-0.630,-0.599,-0.126,-0.314,-0.544,-0.192
6219,1373158606,-0.006,-0.613,0.495,-0.735,-0.644,0.518,-0.006,-0.605,0.429,-0.504,-0.954,0.656,-0.028,-0.547,0.236,-0.527,0.525,-0.192,-0.014,-0.350,0.572,0.432,0.253,0.154,0.512,-0.459,0.385,0.669,0.769,-0.136,0.000,0.000,-0.003,-0.233,-0.005,-0.011,-0.188,0.000,-0.313,-0.455,-0.003,0.000,-0.392,-0.554,-0.587,-0.758,0.220,0.268,-0.419,-0.630,-0.912,-0.414,-0.381,-0.544,-0.009
9738,1373158606,-0.674,0.416,-0.721,0.245,0.736,-0.931,-0.589,-0.094,-0.000,0.000,-0.000,0.000,-0.030,-0.552,0.257,-0.574,0.571,-0.209,-0.014,-0.350,0.357,0.189,0.491,0.396,0.287,-0.459,0.136,0.397,0.552,-0.136,0.000,0.000,0.194,0.199,0.000,0.000,0.196,0.000,0.201,0.183,0.189,0.000,-0.392,-0.554,-0.459,-0.492,0.474,0.533,-0.226,-0.630,-0.551,-0.038,-0.229,-0.544,-0.196
2916,1373158606,-0.040,0.209,-0.041,0.869,-0.519,-0.452,-0.046,0.422,-1.849,6.610,-8.081,3.039,0.025,-0.285,-0.616,1.374,-1.367,0.500,-0.014,-0.350,0.312,0.368,0.511,0.406,0.416,-0.459,0.347,0.625,0.511,-0.136,0.000,0.000,-0.044,-0.269,-0.004,-0.057,-0.262,0.000,-0.272,-0.191,-0.038,0.000,-0.392,-0.554,-0.320,-0.798,0.475,0.519,-0.397,-0.630,-0.945,0.165,-0.109,-0.544,-0.040
6487,1373158606,-0.125,0.345,-0.093,0.593,0.279,-0.621,-0.127,0.419,-0.831,3.266,-3.650,1.354,0.025,-0.286,-0.612,1.366,-1.359,0.497,-0.014,-0.350,0.285,0.130,0.629,0.436,0.244,-0.459,0.108,0.732,0.478,-0.136,0.000,0.000,-0.072,-0.131,-0.005,-0.111,-0.062,0.000,-0.110,-0.318,0.018,0.000,-0.392,-0.554,-0.304,-0.344,0.592,0.557,-0.049,-0.630,-0.401,0.228,-0.037,-0.544,-0.122


In [18]:
lambda_net_valid_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36
4647,1373158606,0.875,0.572,-0.454,-0.406,-0.861,-0.428,0.894,0.617,-2.790,9.841,-16.357,7.263,0.025,-0.285,-0.617,1.377,-1.370,0.501,-0.014,-0.350,0.769,0.699,0.248,0.147,0.844,-0.459,0.663,0.111,1.010,-0.136,0.000,0.000,-0.407,-0.452,0.367,0.367,-0.621,0.000,-0.520,0.494,-0.362,0.000,-0.392,-0.554,-1.121,-1.351,0.587,0.681,-1.065,-0.630,-1.512,0.263,-0.764,-0.544,0.310
9707,1373158606,-0.011,0.254,-0.681,-0.863,0.070,0.095,-0.005,0.158,-0.194,-1.874,0.951,-0.173,0.012,-0.480,-0.109,0.247,-0.247,0.091,-0.014,-0.350,0.643,0.490,0.198,0.097,0.592,-0.459,0.455,0.731,0.864,-0.136,0.000,0.000,-0.249,-0.356,0.001,0.001,-0.364,0.000,-0.384,-0.379,-0.190,0.000,-0.392,-0.554,-0.777,-0.957,0.170,0.225,-0.674,-0.630,-1.068,-0.465,-0.523,-0.544,-0.002
456,1373158606,0.983,0.089,0.926,-0.849,-0.269,-0.549,0.959,0.800,-4.589,15.748,-21.451,9.053,0.025,-0.284,-0.621,1.386,-1.378,0.504,-0.014,-0.350,0.608,0.543,0.409,0.311,0.673,-0.459,0.364,0.365,0.801,-0.136,0.000,0.000,-0.382,-0.391,0.391,0.389,-0.503,0.000,0.003,0.505,-0.398,0.000,-0.392,-0.554,-0.924,-1.152,0.614,0.692,-0.813,-0.630,-0.704,0.268,-0.585,-0.544,0.329
2842,1373158606,-0.592,-0.938,0.027,-0.831,-0.061,0.866,-0.560,-1.266,0.862,-2.492,2.838,-1.034,-0.030,-0.551,0.257,-0.574,0.571,-0.209,-0.014,-0.350,0.535,0.371,0.291,0.341,0.469,-0.459,0.321,0.577,0.728,-0.136,0.000,0.000,0.155,0.156,0.000,-0.249,0.163,0.000,0.155,0.189,0.160,0.000,-0.392,-0.554,-0.576,-0.573,0.246,0.716,-0.331,-0.630,-0.613,-0.158,-0.365,-0.544,-0.147
7477,1373158606,-0.630,-0.182,0.280,0.955,-0.392,-0.206,-0.633,-0.096,-0.299,2.422,-1.979,0.412,-0.030,-0.549,0.255,-0.569,0.566,-0.207,-0.014,-0.350,0.182,-0.003,0.639,0.512,0.107,-0.459,-0.059,0.682,0.407,-0.136,0.000,0.000,0.210,0.223,-0.207,-0.290,0.210,0.000,0.228,-0.104,0.188,0.000,-0.392,-0.554,-0.403,-0.545,0.645,0.745,-0.201,-0.630,-0.646,0.208,-0.109,-0.544,-0.217


In [19]:
lambda_net_test_dataset_list[-1].as_pandas().head()

,seed,0-target,1-target,2-target,3-target,4-target,5-target,0-lstsq_lambda,1-lstsq_lambda,2-lstsq_lambda,3-lstsq_lambda,4-lstsq_lambda,5-lstsq_lambda,0-lstsq_target,1-lstsq_target,2-lstsq_target,3-lstsq_target,4-lstsq_target,5-lstsq_target,wb_0,wb_1,wb_2,wb_3,wb_4,wb_5,wb_6,wb_7,wb_8,wb_9,wb_10,wb_11,wb_12,wb_13,wb_14,wb_15,wb_16,wb_17,wb_18,wb_19,wb_20,wb_21,wb_22,wb_23,wb_24,wb_25,wb_26,wb_27,wb_28,wb_29,wb_30,wb_31,wb_32,wb_33,wb_34,wb_35,wb_36
7772,1373158606,-0.439,-0.128,0.786,0.309,-0.275,-0.019,-0.435,-0.232,1.563,-1.927,2.394,-1.135,-0.018,-0.421,0.089,-0.200,0.200,-0.073,-0.014,-0.350,0.160,-0.026,0.666,0.515,0.158,-0.459,-0.090,0.836,0.432,-0.136,0.000,0.000,0.193,0.200,-0.132,-0.329,0.160,0.000,0.205,-0.347,0.159,0.000,-0.392,-0.554,-0.271,-0.375,0.643,0.728,-0.027,-0.630,-0.484,0.386,-0.005,-0.544,-0.210
9620,1373158606,0.194,0.764,-0.831,-0.543,-0.365,0.546,0.240,0.166,1.783,-5.652,4.219,-0.994,0.025,-0.284,-0.621,1.387,-1.380,0.505,-0.014,-0.350,0.547,0.396,0.422,0.324,0.514,-0.459,0.335,0.563,0.754,-0.136,0.000,0.000,-0.235,-0.250,0.129,0.129,-0.276,0.000,-0.256,-0.148,-0.234,0.000,-0.392,-0.554,-0.690,-0.765,0.420,0.473,-0.495,-0.630,-0.824,-0.144,-0.452,-0.544,0.121
3656,1373158606,-0.021,-0.821,0.493,0.695,-0.784,0.633,-0.033,-0.703,0.551,-1.229,3.017,-1.458,-0.019,-0.422,0.101,-0.227,0.227,-0.084,-0.014,-0.350,0.427,0.263,0.697,0.634,0.365,-0.459,0.212,0.834,0.623,-0.136,0.000,0.000,0.013,0.013,-0.387,-0.487,0.007,0.000,0.013,-0.254,0.009,0.000,-0.392,-0.554,-0.432,-0.416,0.969,1.252,-0.183,-0.630,-0.448,0.496,-0.229,-0.544,-0.017
5979,1373158606,-0.168,-0.063,0.254,0.990,0.200,0.853,-0.184,-0.086,2.561,-10.219,18.287,-8.548,0.024,-0.294,-0.571,1.276,-1.270,0.465,-0.014,-0.350,0.185,0.139,1.097,0.996,0.486,-0.459,0.113,1.469,0.717,-0.136,0.000,0.000,-0.186,-0.141,-0.445,-0.740,-0.368,0.000,-0.117,-0.924,0.027,0.000,-0.392,-0.554,-0.245,-0.348,1.200,1.490,0.440,-0.630,-0.401,1.041,0.288,-0.544,-0.200
2521,1373158606,0.281,-0.063,0.716,-0.315,-0.467,0.970,0.264,-0.075,2.491,-8.299,11.764,-5.189,0.025,-0.284,-0.622,1.387,-1.380,0.505,-0.014,-0.350,0.321,0.127,0.462,0.365,0.563,-0.459,0.104,1.187,0.549,-0.136,0.000,0.000,-0.001,-0.127,0.134,0.134,-0.215,0.000,-0.104,-0.782,-0.003,0.000,-0.392,-0.554,-0.306,-0.353,0.452,0.500,0.493,-0.630,-0.403,0.788,-0.148,-0.544,0.128


## Interpretation Network Training

In [ ]:
(history_list, 
scores_list, 

function_values_complete_list, 
function_values_valid_list, 
function_values_test_list, 

inet_preds_list, 
inet_preds_valid_list, 
inet_preds_test_list, 

distrib_dict_list,
model_list) = calculate_interpretation_net_results(lambda_net_train_dataset_list, 
                                                          lambda_net_valid_dataset_list, 
                                                          lambda_net_test_dataset_list)

[Parallel(n_jobs=11)]: Using backend MultiprocessingBackend with 11 concurrent workers.


## Evaluate Interpretation Net

In [ ]:
if nas:
    for trial in history_list[-1]: 
        print(trial.summary())

In [ ]:
if len(model_list) >= 1:
    print(model_list[-1].summary())
    print(model_list[-1].get_config())

In [ ]:
scores_list[-1]

In [ ]:
distrib_dict_list[-1]['MAE']

In [ ]:
distrib_dict_list[-1]['R2']

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
polynomial_inet = inet_preds_test_list[-1][index_min]

print(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][index_min])

print_polynomial_from_coefficients(polynomial_inet)

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial

print(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][index_min])

polynomial_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index_min]
print_polynomial_from_coefficients(polynomial_lambda)

In [ ]:
index_min = int(np.argmin(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial
polynomial_target = lambda_net_test_dataset.target_polynomial_list[index_min]
print_polynomial_from_coefficients(polynomial_target)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
polynomial_inet = inet_preds_test_list[-1][index_max]

print(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'][index_max])

print_polynomial_from_coefficients(polynomial_inet)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial

print(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'][index_max])

polynomial_lambda = lambda_net_test_dataset.lstsq_lambda_pred_polynomial_list[index_max]
print_polynomial_from_coefficients(polynomial_lambda)

In [ ]:
index_max = int(np.argmax(distrib_dict_list[-1]['R2'].loc['predLambda_VS_lstsqTarget_test']))
#polynomial_target = lambda_net_test_dataset.get_lambda_net_by_lambda_index(index_min).target_polynomial
polynomial_target = lambda_net_test_dataset.target_polynomial_list[index_max]
print_polynomial_from_coefficients(polynomial_target)

In [ ]:
r2_values_inet = distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test']
print('Mean: ' + str(np.mean(r2_values_inet)) + ' (' + str(r2_values_inet.shape[0]) + ' Samples)')

r2_values_positive_inet = r2_values_inet[r2_values_inet>0]
print('Mean (only positive): ' + str(np.mean(r2_values_positive_inet)) + ' (' + str(r2_values_positive_inet.shape[0]) + ' Samples)')



In [ ]:
r2_values_lstsq_lambda = distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test']
print('Mean: ' + str(np.mean(r2_values_lstsq_lambda)) + ' (' + str(r2_values_inet.shape[0]) + ' Samples)')

r2_values_positive_lstsq_lambda = r2_values_lstsq_lambda[r2_values_lstsq_lambda>0]
print('Mean (only positive): ' + str(np.mean(r2_values_positive_lstsq_lambda)) + ' (' + str(r2_values_positive_lstsq_lambda.shape[0]) + ' Samples)')



In [ ]:
sns.histplot(distrib_dict_list[-1]['MAE'].loc['inetPoly_VS_targetPoly_test'])


In [ ]:
sns.histplot(distrib_dict_list[-1]['MAE'].loc['lstsqLambda_VS_targetPoly_test'])


In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'], binwidth=0.2)
p.set(xlim=(-30, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['inetPoly_VS_targetPoly_test'], binwidth=0.1)
p.set(xlim=(0, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'], binwidth=0.2)
p.set(xlim=(-10, 1))

In [ ]:
p = sns.histplot(distrib_dict_list[-1]['R2'].loc['lstsqLambda_VS_targetPoly_test'], binwidth=0.1)
p.set(xlim=(0, 1))

In [ ]:
if not nas:
    history = history_list[-1]

    plt.plot(history[list(history.keys())[1]])
    if consider_labels_training or evaluate_with_real_function:
        plt.plot(history[list(history.keys())[len(history.keys())//2+1]]) 
    plt.title('model ' + list(history.keys())[1])
    plt.ylabel('metric')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('./data/results/' + path_identifier_interpretation_net_data + '/metric_' + path_identifier_interpretation_net_data + '_epoch_' + str(epochs_lambda).zfill(3) + '.png')


In [ ]:
if not nas:
    history = history_list[-1]

    plt.plot(history['loss'])
    if consider_labels_training or evaluate_with_real_function:
        plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('./data/results/' + path_identifier_interpretation_net_data + '/loss_' + path_identifier_interpretation_net_data + '_epoch_' + str(epochs_lambda).zfill(3) + '.png')    


### Multi Epoch/Sampes Analysis

### Generate Comparison Plots

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['MAE FV', 'RMSE FV', 'MAPE FV', 'R2 FV', 'RAAE FV', 'RMAE FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list)

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['MAE FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list)

In [ ]:
if len(scores_list) > 1:
    plot_metric_list = ['R2 FV']

    generate_inet_comparison_plot(scores_list, plot_metric_list, ylim=(-5, 1))

#### Generate and Analyze Predictions for Random Function

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      rand_index=1, 
                                                      plot_type=1)

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      rand_index=1, 
                                                      plot_type=2)

In [ ]:
plot_and_save_single_polynomial_prediction_evaluation(lambda_net_test_dataset_list, 
                                                      function_values_test_list, 
                                                      rand_index=1, 
                                                      plot_type=3)

# BENCHMARK (RANDOM GUESS) EVALUATION

In [ ]:
list_of_random_polynomials = np.random.uniform(low=-10, high=10, size=(len(lambda_net_test_dataset_list[-1]), sparsity))

In [ ]:
true_fv_test = parallel_fv_calculation_from_polynomial(lambda_net_test_dataset_list[-1].target_polynomial_list, lambda_net_test_dataset_list[-1].test_data_list)
random_fv_test = parallel_fv_calculation_from_polynomial(list_of_random_polynomials, lambda_net_test_dataset_list[-1].test_data_list)

In [ ]:
print('Random Guess Error Coefficients: ' + str(np.round(mean_absolute_error(lambda_net_test_dataset_list[-1].target_polynomial_list, list_of_random_polynomials), 4)))

In [ ]:
print('Random Guess Error FVs: ' + str(np.round(mean_absolute_error_function_values(true_fv_test, random_fv_test), 4)))

# BENCHMARK (EDUCATED GUESS/MEAN PREDICTION) EVALUATION

In [ ]:
true_fv_train = parallel_fv_calculation_from_polynomial(lambda_net_test_dataset_list[-1].target_polynomial_list, lambda_net_test_dataset_list[-1].test_data_list)

mean_fv = np.mean(true_fv_train)
mean_fv_pred_test = [mean_fv for _ in range(true_fv_test.shape[0])]

In [ ]:
print('Educated Guess/Mean Prediction Error FVs: ' + str(np.round(mean_absolute_error_function_values(true_fv_test, mean_fv_pred_test), 4)))

In [ ]:
%%script false --no-raise-error

base_model = generate_base_model()
random_evaluation_dataset = np.random.uniform(low=x_min, high=x_max, size=(random_evaluation_dataset_size, n))
#random_evaluation_dataset = lambda_train_input_train_split[0]#lambda_train_input[0] #JUST [0] HERE BECAUSE EVALUATION ALWAYS ON THE SAME DATASET FOR ALL!!
list_of_monomial_identifiers_numbers = np.array([list(monomial_identifiers) for monomial_identifiers in list_of_monomial_identifiers]).astype(float)


loss_function = mean_absolute_error_tf_fv_lambda_extended_wrapper(random_evaluation_dataset, list_of_monomial_identifiers_numbers, base_model)      

X_train = X_train_list[-1].values[:,1:]
y_train = y_train_list[-1].values[:,2:]

#X_train = X_train[:,1:]
y_train_model = np.hstack((y_train, X_train))

print('seed_in_inet_training = ' + str(seed_in_inet_training), loss_function(y_train_model, y_train))


seed_in_inet_training = False

loss_function = mean_absolute_error_tf_fv_lambda_extended_wrapper(random_evaluation_dataset, list_of_monomial_identifiers_numbers, base_model)      

X_train = X_train_list[-1].values[:,1:]
y_train = y_train_list[-1].values[:,2:]

X_train = X_train[:,1:]
y_train_model = np.hstack((y_train, X_train))

print('seed_in_inet_training = ' + str(seed_in_inet_training), loss_function(y_train_model, y_train))

seed_in_inet_training = True



In [ ]:
if use_gpu:
    from numba import cuda 
    device = cuda.get_current_device()
    device.reset()